In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
# Uploading all professor modules
import sys
sys.path.append('../input/notebook2')
print(sys.version)
!cp -r ../input/notebook2/* ./

In [ ]:
#cython
!python run_compile_all_cython.py

In [ ]:
def load_URM(file_path):
    import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

    data = pd.read_csv(file_path)

    import scipy.sparse as sps

    user_list = data['row'].tolist()
    item_list = data['col'].tolist()
    rating_list = data['data'].tolist()

    return sps.coo_matrix((rating_list, (user_list, item_list))).tocsr()

In [ ]:
#split URM train e validation
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM('../input/recsys/data_train.csv')

In [ ]:
def load_ICM(file_path):
    import pandas as pd
    import scipy.sparse as sps

    metadata = pd.read_csv(file_path)

    item_icm_list = metadata['row'].tolist()
    feature_list = metadata['col'].tolist()
    weight_list = metadata['data'].tolist()

    return sps.coo_matrix((weight_list, (item_icm_list, feature_list)))

In [ ]:
import scipy.sparse as sps
import numpy as np

ICM_channel = load_ICM('../input/recsys/data_ICM_channel.csv')
ICM_event = load_ICM('../input/recsys/data_ICM_event.csv')
ICM_genre = load_ICM('../input/recsys/data_ICM_genre.csv')
ICM_subgenre = load_ICM('../input/recsys/data_ICM_subgenre.csv')


In [ ]:
new_ICM = sps.hstack([ICM_channel, ICM_genre, ICM_subgenre])

In [ ]:
#stacking on urml_all given the fact I'll fit it
stacked_URM = sps.vstack([URM_all, new_ICM.T])
stacked_URM = sps.csr_matrix(stacked_URM)

stacked_ICM = sps.csr_matrix(stacked_URM.T)

stacked_URM, stacked_ICM

In [ ]:
#now urm is enriched with icm
URM_train, URM_validation = split_train_in_two_percentage_global_sample(stacked_URM, train_percentage = 0.8)

Hybrids

In [ ]:
#best found ials with icm
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

reciALS= IALSRecommender(stacked_URM) #stackeeeeeeeeed
reciALS.fit(epochs = 15,
            num_factors = 45,
            confidence_scaling = "log",
            alpha = 0.5170311361186297,
            epsilon = 0.40292557422959563,
            reg = 0.004544677284256727
            #init_mean=0.0,
            #init_std=0.1
            #**earlystopping_kwargs
               )


In [ ]:
#best found slimen with icm
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recSlimEN= SLIMElasticNetRecommender(stacked_URM)
recSlimEN.fit(l1_ratio=1.1513783084985463e-05, alpha = 0.03279093531141528, positive_only=True, topK = 1386)


In [ ]:
#saving slimen

def save_model(self, folder_path, file_name = None):
        raise NotImplementedError("BaseRecommender: save_model not implemented")
        
def load_model(self, folder_path, file_name = None):

        if file_name is None:
            file_name = self.RECOMMENDER_NAME

        self._print("Loading model from file '{}'".format(folder_path + file_name))

        dataIO = DataIO(folder_path=folder_path)
        data_dict = dataIO.load_data(file_name=file_name)

        for attrib_name in data_dict.keys():
             self.__setattr__(attrib_name, data_dict[attrib_name])

        self._print("Loading complete")

In [ ]:
import numpy as np
from Recommenders.DataIO import DataIO
from Recommenders.Recommender_utils import check_matrix

In [ ]:
#save_model(recSlimEN, folder_path = "./" ,file_name = 'slimen2281')
#load_model(self, folder_path, file_name = None)

In [ ]:
ratings = pd.read_csv('../input/recsys/data_train.csv').rename(columns = {'row':'user','col':'item','data':'rating'})
ratings

In [ ]:
ratings.apply(set)

In [ ]:
users_id = list(ratings.apply(set)[0])

In [ ]:
item_scores_slimen = recSlimEN._compute_item_score(users_id)
item_scores_slimen

In [ ]:
item_scores_ials = reciALS._compute_item_score(users_id)
item_scores_ials

In [ ]:
#alpha = 0.7

In [ ]:
#new_item_scores = alpha * item_scores_slimen + (1 - alpha) * item_scores_puresvd
#new_item_scores

In [ ]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, alpha):
        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [ ]:
import scipy.sparse as sps

Fitting the hybrid

In [ ]:
scoreshybridrecommender = ScoresHybridRecommender(stacked_URM, recSlimEN, reciALS)
scoreshybridrecommender.fit(alpha = 0.78)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout

cutoff_list=[10]

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list, exclude_seen = False)

In [ ]:
result_df, _ = evaluator_validation.evaluateRecommender(scoreshybridrecommender)
result_df

In [ ]:
 def write_submission(recommender, target_users_path,out_path):
    import pandas as pd
    import csv
    import numpy as np

    targetUsers = pd.read_csv(target_users_path)['user_id']


    targetUsers = targetUsers.tolist()

    with open(out_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['user_id', 'item_list'])

        for userID in targetUsers:
            writer.writerow([userID, str(np.array(recommender.recommend(userID, 10)))[1:-1]])

In [ ]:
write_submission(recommender=scoreshybridrecommender, target_users_path="../input/recsys/data_target_users_test.csv",
                     out_path='output_h_slimenicm+ialsicm.csv')